# have vs love vs want

[stacked bar(px)](https://plotly.com/python/bar-charts/)

## 1. importar las librerías + csv con los datos de la encuesta. 

In [1]:
# importar librerias

import pandas as pd
import plotly.express as px  
import plotly.graph_objects as go

import webbrowser
from threading import Timer
import numpy as np
import dash
from dash import Dash, dcc, html, Input, Output, State
import dash_bootstrap_components as dbc #bootstrap


from collections import Counter
from itertools import chain
import time
df21 = pd.read_csv ('../../data/survey_results_public2021.csv', index_col = [0]) # El indice sera la columna con el ID de la respuesta


## 2. Preprocesar datos.

Tratar las columnas/conjunto de datos para comenzar a crear los gráficos. 

In [2]:
# copiamos elementos que no sean NaN


def loveHateWant (df, columna1, columna2):
    #df2=df[['DatabaseHaveWorkedWith','DatabaseWantToWorkWith']].dropna().copy()
    df2=df[[columna1, columna2]].dropna().copy()
    
    df3=pd.DataFrame()


    df3['love'] = [set(x[0].split(';')) & set(x[1].split(';')) for x in df2.values]
    df3['hate'] = [set(x[0].split(';')) - set(x[1].split(';')) for x in df2.values]
    df3['want'] = [set(x[1].split(';')) - set(x[0].split(';')) for x in df2.values] 


    df_counts = (pd.DataFrame([Counter(chain.from_iterable(df3[column]))
                    for column in df3.columns],
                    index=['love', 'hate', 'want'])
                    .fillna(0)
                    .T
                    .sort_index()
)

    
    return df_counts


## 3. Grafico. 

En este caso, un diagrama de barras.

In [3]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP],
                meta_tags=[{'name': 'viewport', #permite ser responsive en movil
                            'content': 'width=device-width, initial-scale=1.0, maximum-scale=1.2, minimum-scale=0.5,'}],
                #long_callback_manager=long_callback_manager
               )

app.layout = html.Div([
    
#     html.Div( id = "div-loading",
             
#              children=[
    
#     #spiner
#        #color = "#FFFF" se puede añadir un hex color sise quiere
#             dbc.Spinner( size="lg", color="primary", type="border", 
#                         fullscreen=True, id="initial-spinner"),
#             # spinner_style={"width": "10rem", "height": "10rem"}), esto lo hace mas grande, pero hay que quitar el 
#           #parametro size para que no se contradigan
#             # spinnerClassName="spinner"), #añadir css desde assets. No en mi caso.
#             # dcc.Loading(children=[dcc.Graph(id="loading-output")], color="#119DFF", type="dot", fullscreen=True,),

#             #width={'size': 12, 'offset': 0}),
#         ]
#             ),
    
    #spiner
             
    
    html.Div(
                 className="div-app",
                 id="div-app",
                 children = [ #whole app layout here.
                     
    
    
                dbc.Row([
                        dbc.Tabs([
                            dbc.Tab(label='Databases', tab_id='db',labelClassName="text-primary font-weight-bold", activeLabelClassName="text-info"),
                            dbc.Tab(label='Languages', tab_id='lang', labelClassName="text-primary font-weight-bold", activeLabelClassName="text-info"), 
                            dbc.Tab(label='Platform', tab_id='plat', labelClassName="text-primary font-weight-bold", activeLabelClassName="text-info"), 
                            dbc.Tab(label='webframe', tab_id='web', labelClassName="text-primary font-weight-bold", activeLabelClassName="text-info"), 
                            dbc.Tab(label='Miscelaneous', tab_id='misc', labelClassName="text-primary font-weight-bold", activeLabelClassName="text-info"), 
                            dbc.Tab(label='Tools', tab_id='tools', labelClassName="text-primary font-weight-bold", activeLabelClassName="text-info"), 
                            dbc.Tab(label='Collab', tab_id='colab', labelClassName="text-primary font-weight-bold", activeLabelClassName="text-info"),
                            ],
                            id="tabs",
                            active_tab="db",
                        ),
                    #dcc.Loading([
                    dbc.CardBody(html.P(id="tab-content", className="card-text")),
                    #]),
            ])
                     
                 ])
])




In [4]:

#tabs

@app.callback(
    output=Output("tab-content", "children"),
    inputs=Input("tabs", "active_tab"),
#     manager=long_callback_manager,
#     prevent_initial_call=True,
)
    
def tab_content(active_tab):
    
    df = pd.DataFrame()
    
    nameColumns= ['DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith', 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith',
   'PlatformHaveWorkedWith', 'PlatformWantToWorkWith', 'WebframeHaveWorkedWith', 'WebframeWantToWorkWith',
   'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith', 'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith',
   'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantToWorkWith']
    #valores para iterar en las posiciones del array namColumns
    col1 = 0
    col2 = 0

    
    name = ["Databases", "Languages", "Platforms", "Webframes", "Miscelaneous Tech", "Tools", "Collab tools"]
    orden = 0 
    
    if active_tab is not None: #para evitar un error hay que añadir un caso donde active_tab sste vacio
        if active_tab =="db":
            col1 = 0 # posicion de columna en namecolumns : DatabaseHaveWorkedWith
            col2 = 1
            orden = 0 #posicion de name : Databases
            
        elif active_tab == "lang":
            col1 = 2
            col2 = 3
            orden = 1
            
        elif active_tab == "plat":
            col1 = 4
            col2 = 5
            orden = 2
            
        elif active_tab == "web":
            col1 = 6
            col2 = 7
            orden = 3
            
        elif active_tab == "misc":
            col1 = 8
            col2 = 9
            orden = 4
            
        elif active_tab == "tools":
            col1 = 10
            col2 = 11
            orden = 5
            
        elif active_tab == "colab":
            col1 = 12
            col2 = 13
            orden = 6
        
        df = loveHateWant (df21, nameColumns[col1], nameColumns[col2])
        
        fig = px.bar(df, x = ['love', 'hate', 'want'], y = df.index,               
                 #template = 'plotly_dark',
                 color_discrete_sequence = px.colors.qualitative.T10, 
                 title = 'Love Vs Hate Vs Want:'+ name[orden],
                 #title = 'Love Vs Hate Vs Want: Collab tools',
                 orientation = 'h',
            )
        
        #nombre de ejes
        fig.update_yaxes(title_text=name[orden])
        fig.update_xaxes(title_text="# Responses")
            
        #quitar color y grid del grafico
        fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',}) #fondo transparente
        fig.update_layout(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False)) #eliminar grid
            
        return dcc.Graph(figure=fig, style={'height': '90vh'}) 
        
        
    return html.P("This shouldn't ever be displayed...")
    
    

## 4. run server

In [ ]:
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
